In [1]:
from pathlib import Path
import json, os, re, subprocess, shutil, zipfile, tarfile
from urllib.parse import urlparse

def download_datasets_from_json(
    json_path,
    root="/home/kristoffel/datasets",
    force=False,
    unzip=True,
    http_timeout=60,
):
    import requests  # used only for direct HTTP downloads

    # ---- small helpers ----
    def slugify(text: str) -> str:
        text = text.lower()
        text = re.sub(r"&", " and ", text)
        text = re.sub(r"[^a-z0-9]+", "-", text).strip("-")
        return re.sub(r"-{2,}", "-", text)

    def ensure_dir(p: Path): p.mkdir(parents=True, exist_ok=True)
    def is_nonempty_dir(p: Path) -> bool: return p.exists() and any(p.iterdir())

    def write_readme(folder: Path, name: str, urls: list[str], extra: str = ""):
        ensure_dir(folder)
        readme = folder / "README.txt"
        lines = [name, "", "Links:"]
        for u in urls: lines.append(f"- {u}")
        if extra: lines += ["", extra]
        readme.write_text("\n".join(lines) + "\n", encoding="utf-8")
        print(f"📝 Wrote {readme}")

    def unzip_all_in_dir(d: Path):
        for z in sorted(d.glob("*.zip")):
            print(f"📦 Unzipping {z.name}")
            with zipfile.ZipFile(z, "r") as zf: zf.extractall(d)
            z.unlink()
        tars = list(d.glob("*.tar")) + list(d.glob("*.tar.gz")) + list(d.glob("*.tgz")) \
             + list(d.glob("*.tar.bz2")) + list(d.glob("*.tbz2")) + list(d.glob("*.tar.xz"))
        for t in sorted(tars):
            print(f"📦 Extracting {t.name}")
            with tarfile.open(t, "r:*") as tf: tf.extractall(d)
            t.unlink()

    def kaggle_info_from_url(url: str):
        p = urlparse(url)
        if "kaggle.com" not in (p.netloc or ""): return None, None
        parts = [x for x in (p.path or "").split("/") if x]
        if not parts: return None, None
        if parts[0] == "datasets" and len(parts) >= 3:
            return "dataset", f"{parts[1]}/{parts[2]}"
        if parts[0] == "competitions" and len(parts) >= 2:
            return "competition", parts[1]
        return None, None

    def have_kaggle_cli():
        try:
            cp = subprocess.run(["kaggle", "--version"], capture_output=True, text=True)
            return cp.returncode == 0
        except FileNotFoundError:
            return False

    def have_kagglehub():
        try:
            import kagglehub  # noqa: F401
            return True
        except Exception:
            return False

    def kagglehub_dataset_download(slug: str, dest: Path) -> bool:
        try:
            import kagglehub
            print(f"⬇️ KaggleHub dataset: {slug}")
            cache_path = Path(kagglehub.dataset_download(slug))
            print(f"   cache at: {cache_path}")
            # copy cache into destination
            ensure_dir(dest)
            shutil.copytree(cache_path, dest, dirs_exist_ok=True)
            return True
        except Exception as e:
            print(f"❌ KaggleHub failed: {e}")
            return False

    def kaggle_cli_download(mode: str, slug: str, dest: Path) -> bool:
        ensure_dir(dest)
        if mode == "dataset":
            cmd = ["kaggle", "datasets", "download", "-d", slug, "-p", str(dest), "-o"]
        else:
            cmd = ["kaggle", "competitions", "download", "-c", slug, "-p", str(dest), "-o"]
        print("→", " ".join(cmd))
        cp = subprocess.run(cmd, capture_output=True, text=True)
        print(cp.stdout or "")
        if cp.returncode != 0:
            if cp.stderr: print(cp.stderr)
            print("❌ Kaggle CLI download failed (make sure creds exist, and accept comp rules if needed).")
            return False
        if unzip: unzip_all_in_dir(dest)
        return True

    def http_download(url: str, dest_dir: Path) -> Path | None:
        name = os.path.basename(urlparse(url).path) or "download.bin"
        out = dest_dir / name
        print(f"⬇️ HTTP: {url}\n   → {out}")
        try:
            with requests.get(url, stream=True, timeout=http_timeout) as r:
                r.raise_for_status()
                with open(out, "wb") as f:
                    for chunk in r.iter_content(1024 * 1024):
                        if chunk: f.write(chunk)
            return out
        except Exception as e:
            print(f"❌ HTTP download failed: {e}")
            return None

    def maybe_direct_download(url: str, dest: Path) -> bool:
        lower = url.lower()
        looks_file = any(lower.endswith(ext) for ext in
                         (".zip", ".tar", ".tgz", ".tar.gz", ".tar.bz2", ".tbz2", ".tar.xz"))
        if not looks_file: return False
        fp = http_download(url, dest)
        if fp and unzip: unzip_all_in_dir(dest)
        return fp is not None

    # ---- load JSON + prep root ----
    json_path = Path(json_path).expanduser().resolve()
    data = json.loads(json_path.read_text(encoding="utf-8"))
    items = data.get("datasets", [])
    root = Path(root).expanduser().resolve()
    ensure_dir(root)

    kh_ok = have_kagglehub()
    cli_ok = have_kaggle_cli()
    if not kh_ok: print("ℹ️ kagglehub not installed; will try Kaggle CLI or fallback to README.")
    if not cli_ok: print("ℹ️ Kaggle CLI not available; competitions will need manual/README unless KaggleHub supports them.")

    created = []
    for d in sorted(items, key=lambda x: x.get("id", 0)):
        did, name, urls = d.get("id"), d.get("name", f"dataset_{d.get('id')}"), d.get("urls", [])
        target = root / f"dataset{did}_{slugify(name)}"
        ensure_dir(target)

        if is_nonempty_dir(target) and not force:
            print(f"✅ Exists (skipping): {target}")
            created.append(target)
            continue

        print(f"\n=== [{did}] {name} → {target} ===")

        success = False

        # Prefer KaggleHub for Kaggle *datasets*
        for u in urls:
            mode, slug = kaggle_info_from_url(u)
            if mode == "dataset" and kh_ok:
                success = kagglehub_dataset_download(slug, target)
                if success: break

        # If KaggleHub didn't run (or it failed), try Kaggle CLI (datasets/competitions)
        if not success:
            for u in urls:
                mode, slug = kaggle_info_from_url(u)
                if mode and cli_ok:
                    success = kaggle_cli_download(mode, slug, target)
                    if success: break

        # Try direct HTTP for archive-looking links
        if not success:
            for u in urls:
                if maybe_direct_download(u, target):
                    success = True
                    break

        # If still not downloaded, leave instructions
        if not success:
            extra = ("Manual step required. Download from the links above (may need account/approval), "
                     "place files here, and unzip in this folder.")
            write_readme(target, name, urls, extra=extra)

        created.append(target)

    print("\n🎉 Done.")
    return created


In [4]:
# If you haven't installed kagglehub yet (do once):
# %pip install kagglehub

download_datasets_from_json(
    json_path="/home/kristoffel/datasets/datasets.json",
    root="/home/kristoffel/datasets",
    force=False,
    unzip=True
)


ℹ️ Kaggle CLI not available; competitions will need manual/README unless KaggleHub supports them.

=== [1] Nutrition5K → /home/kristoffel/datasets/dataset1_nutrition5k ===
📝 Wrote /home/kristoffel/datasets/dataset1_nutrition5k/README.txt
✅ Exists (skipping): /home/kristoffel/datasets/dataset2_food101
✅ Exists (skipping): /home/kristoffel/datasets/dataset3_food11
✅ Exists (skipping): /home/kristoffel/datasets/dataset4_foodd
✅ Exists (skipping): /home/kristoffel/datasets/dataset5_ifood-2019-fgvc6
✅ Exists (skipping): /home/kristoffel/datasets/dataset6_isia-food-500
✅ Exists (skipping): /home/kristoffel/datasets/dataset7_large-scale-food-recognition
✅ Exists (skipping): /home/kristoffel/datasets/dataset8_food-ingredient-dataset-51
✅ Exists (skipping): /home/kristoffel/datasets/dataset9_uecfoodpix-and-uecfoodpixcomplete

🎉 Done.


[PosixPath('/home/kristoffel/datasets/dataset1_nutrition5k'),
 PosixPath('/home/kristoffel/datasets/dataset2_food101'),
 PosixPath('/home/kristoffel/datasets/dataset3_food11'),
 PosixPath('/home/kristoffel/datasets/dataset4_foodd'),
 PosixPath('/home/kristoffel/datasets/dataset5_ifood-2019-fgvc6'),
 PosixPath('/home/kristoffel/datasets/dataset6_isia-food-500'),
 PosixPath('/home/kristoffel/datasets/dataset7_large-scale-food-recognition'),
 PosixPath('/home/kristoffel/datasets/dataset8_food-ingredient-dataset-51'),
 PosixPath('/home/kristoffel/datasets/dataset9_uecfoodpix-and-uecfoodpixcomplete')]